In [7]:
import numpy as np
import scipy.stats as st
import scipy.interpolate as interp
import quantecon as qe
import matplotlib.pyplot as plt

def expect_loss_choose_0(p, L0):
    "For a given probability return expected loss of choosing hypothesis 0"
    return (1-p)*L0

def expect_loss_choose_1(p, L1):
    "For a given probability return expected loss of choosing hypothesis 1"
    return p*L1

def EV(p, f0, f1, V):
    """
    Evaluates the expectation of our value function V. To do this, we
    need the current probability that model 0 is correct (p), the
    distributions (f0, f1), and the function J.
    """
    # Get the current distribution we believe (p*f0 + (1-p)*f1)
    curr_dist = p*f0 + (1-p)*f1
    
    # Get tomorrow's expected distribution through Bayes law
    tp1_dist = np.clip((p*f0) / (p*f0 + (1-p)*f1), 0, 1)
    
    # Evaluate the expectation
    EV = curr_dist @ V(tp1_dist)
    
    return EV

def expect_loss_cont(p, c, f0, f1, V):
    return c + EV(p, f0, f1, V)


def bellman_operator(pgrid, c, f0, f1, L0, L1, V):
    """
    Evaluates the value function for a given continuation value
    function; that is, evaluates

        V(p) = min((1 - p) L0, p L1, c + E V(p'))

    Uses linear interpolation between points.
    """
    m = np.size(pgrid)
    assert m == np.size(V)
    
    V_out = np.zeros(m)
    V_interp = interp.UnivariateSpline(pgrid, V, k=1, ext=0)

    for (p_ind, p) in enumerate(pgrid):
        # Payoff of choosing model 0
        p_c_0 = expect_loss_choose_0(p, L0)
        p_c_1 = expect_loss_choose_1(p, L1)
        p_con = expect_loss_cont(p, c, f0, f1, V_interp)
        
        V_out[p_ind] = min(p_c_0, p_c_1, p_con)

    return V_out

#  == Now run at given parameters == #


pg = np.linspace(0, 1, 200)

# initial value

V=100*np.ones(200)

m=200
f0=np.linspace(0, 1, 2)
f1=np.linspace(0, 1, 2)
f0[0]=1/4
f0[1]=3/4
f1[0]=2/3
f1[1]=1/3

n = 25

for i in range(n):
    V = bellman_operator(pg, 1, f0, f1, 10, 5, V)

p_c_0= (1-pg)*10
p_c_1= pg*5

lb = pg[np.searchsorted(p_c_1 - V, 1e-10) - 1]
ub = pg[np.searchsorted(V - p_c_0, -1e-10)]

# Simulate Optimal Policy

ndraws=5000

tdist = np.empty(ndraws, int)
cdist = np.empty(ndraws, int)

# Simulate the true distribution is f0

for i in range(ndraws):           
    drv = qe.discrete_rv.DiscreteRV(f0)
    p=1/2
    decision_made = False
    t = 0
    while decision_made is False:
            # the draws come from the true distribution
            k = drv.draw()[0]
            t = t+1
            p = np.clip(p*f0[k] / (p*f0[k] + (1-p)*f1[k]), 0, 1)
            if p < lb:
                decision_made = True
                decision = 1
            elif p > ub:
                decision_made = True
                decision = 0
    tdist[i] = t
    cdist[i] = decision

# Mean and Std Dev of stopping times
mean=np.mean(tdist)
std=np.std(tdist)
  
# Error Rate
Error_rate=np.sum(cdist)/ndraws

In [8]:
mean

1.7354

In [9]:
mean

1.7354

In [10]:
Error_rate

0.4402

In [11]:
std

0.4411199836779105